In [64]:
import requests
import time

In [1]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [66]:
data = requests.get(standings_url)

In [67]:
from bs4 import BeautifulSoup

In [68]:
soup = BeautifulSoup(data.text)
time.sleep(10)

In [69]:
standings_table = soup.select('table.stats_table')[0]

IndexError: list index out of range

In [ ]:
links = standings_table.find_all('a')

In [ ]:
links = [l.get("href") for l in links]

In [ ]:
links = [l for l in links if '/squads/' in l]

In [ ]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [ ]:
team_urls

In [ ]:
team_url = team_urls[0]

In [ ]:
data = requests.get(team_urls[0])

In [ ]:
from io import StringIO
import pandas as pd
matches = pd.read_html(StringIO(data.text), match="Scores & Fixtures")[0]

In [ ]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [ ]:
data = requests.get(f"https://fbref.com{links[0]}")
time.sleep(6)

In [ ]:
shooting = pd.read_html(StringIO(data.text), match="Shooting")[0]

In [ ]:
shooting.head()

In [ ]:
shooting.columns = shooting.columns.droplevel()

In [ ]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [ ]:
shooting.shape

In [ ]:
team_data.head()

In [ ]:
years = list(range(2024,2022, -1))
all_matches = []

In [ ]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [ ]:
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(StringIO(data.text), match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(StringIO(data.text), match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(6)

In [70]:
match_df = pd.concat(all_matches)

In [71]:
match_df.columns = [c.lower() for c in match_df.columns]

In [72]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2023-08-11,20:00,Premier League,Matchweek 1,Fri,Away,W,3,0,Burnley,...,Match Report,NaN,17.0,8.0,13.9,0.0,0,0,2024,Manchester City
3,2023-08-19,20:00,Premier League,Matchweek 2,Sat,Home,W,1,0,Newcastle Utd,...,Match Report,NaN,14.0,4.0,17.9,0.0,0,0,2024,Manchester City
4,2023-08-27,14:00,Premier League,Matchweek 3,Sun,Away,W,2,1,Sheffield Utd,...,Match Report,NaN,29.0,9.0,17.3,2.0,0,1,2024,Manchester City
5,2023-09-02,15:00,Premier League,Matchweek 4,Sat,Home,W,5,1,Fulham,...,Match Report,NaN,6.0,4.0,14.8,0.0,1,1,2024,Manchester City
6,2023-09-16,15:00,Premier League,Matchweek 5,Sat,Away,W,3,1,West Ham,...,Match Report,NaN,29.0,13.0,16.4,1.0,0,0,2024,Manchester City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,2024-04-27,15:00,Premier League,Matchweek 35,Sat,Home,W,5,1,Sheffield Utd,...,Match Report,NaN,19.0,6.0,16.0,0.0,1,1,2024,Newcastle United
47,2024-05-04,15:00,Premier League,Matchweek 36,Sat,Away,W,4,1,Burnley,...,Match Report,NaN,22.0,10.0,17.6,1.0,0,1,2024,Newcastle United
48,2024-05-11,15:00,Premier League,Matchweek 37,Sat,Home,D,1,1,Brighton,...,Match Report,NaN,18.0,7.0,15.8,0.0,0,0,2024,Newcastle United
49,2024-05-15,20:00,Premier League,Matchweek 34,Wed,Away,L,2,3,Manchester Utd,...,Match Report,NaN,21.0,7.0,14.6,0.0,0,0,2024,Newcastle United


In [73]:
match_df.to_csv("matches.csv")